## 1.导入包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from sklearn.model_selection import train_test_split
import time

np.set_printoptions(threshold=np.inf)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
1.功能：
通过加载data_64x64x2.txt文件和label_a_2x64x64x2.txt文件，
将数据预处理将文本文件转换为模型要求的格式，然后继续训练和预测。
定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。
输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）
输出是n*64*64*4
"""


'\n1.功能：\n通过加载data_64x64x2.txt文件和label_a_2x64x64x2.txt文件，\n将数据预处理将文本文件转换为模型要求的格式，然后继续训练和预测。\n定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。\n输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）\n输出是n*64*64*4\n'

## 2. 数据预处理

In [5]:
## 从文件中读取数据并转换为三维格式
print("Start data preprocessing……")


"""
从文件中读取数据并转换为三维列表格式
将6*2*64*64*2的数据转为12*64*64*2,再将其转为6*64*64*4
其中4是前面两个是竖着的a值，后面两个是横着的a值，用于作为标签
"""
def get_label():
    input_file = 'data/label_centerFace_2x64x64x2.txt'
    all_label = []  # 存放所有数据
    single_file_label = []  # 存放一个文件的数据
    i = 0  # 计数，计每个文件的行个数，每4096行保存一次
    with open(input_file, 'r') as f:
        lines = f.readlines()

        # 匹配浮点数和科学计数法表示的浮点数的正则表达式
        pattern = r"[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?"

        matches = re.findall(pattern, str(lines))

        result = [float(match) for match in matches]

        for a in range(len(result)):#716333056
            if a == 0 or a % (4096 * 2) != 0:
                single_file_label.append(result[a])

            else:
                # print(len(single_file_label))#8192
                single_file_label = np.array(single_file_label).reshape(64, 64, 2)
                all_label.append(single_file_label)
                single_file_label = []
                single_file_label.append(result[a])
        single_file_label = np.array(single_file_label).reshape(64, 64, 2)
        all_label.append(single_file_label)
        all_label = np.array(all_label)

        two_label = []
        all_4d_label = []
        num = 0
        for i in all_label:
            num += 1
            two_label.append(i)
            if num % 2 == 0:
                two_label = np.array(two_label)
                two_label = torch.tensor(two_label)
                two_label = two_label.permute(1, 2, 3, 0)
                two_label = torch.reshape(two_label, (two_label.size(0), two_label.size(1), -1))

                # 调整第三维元素的列的顺序
                column_indices = [0, 2, 1, 3]
                two_label = two_label[:, :, column_indices]

                all_4d_label.append(two_label)
                two_label = []
        all_4d_label = np.array([i.numpy() for i in all_4d_label])
        
        print("所有样本的标签处理完毕！")
        return all_4d_label
    


all_label = get_label()  # 获取所有标签


all_label = torch.tensor(all_label).float()

all_label = all_label.permute(0, 3, 1, 2)  # 调整维度顺序，将第4维插入到第2维
"""
维度转换：
(6, 64, 64, 2)
(6, 64, 64, 4)
转为
torch.Size([6, 2, 64, 64])
torch.Size([6, 4, 64, 64])
"""

print("all_label.shape:",all_label.shape)
# all_data.shape: torch.Size([192000, 2, 64, 64])
# all_label.shape: torch.Size([192000, 4, 64, 64])


Start data preprocessing……
所有样本的标签处理完毕！
all_label.shape: torch.Size([192000, 4, 64, 64])


## 3.保存数据

In [6]:
# 将数据写入文件，格式只能为二进制
# # 将数据写入文件，格式只能为二进制
np.save('./data/all_centerFace_label.npy', all_label)
# #直接加载npy文件为numpy格式
# all_label = np.load('./data/all_label.npy')
print("all data has been saved and the format is numpy!")

all data has been saved and the format is numpy!


In [7]:
# 直接加载npy文件为numpy格式
#当数据预处理的代码运行完保存处理后的数据后，可直接运行以下代码加载数据
# all_data = np.load('./data/all_data.npy')
all_label = np.load('./data/all_centerFace_label.npy')

# all_data = torch.tensor(all_data).float()
# all_label = torch.tensor(all_label).float()